In [4]:
import xml.etree.ElementTree as et
import os


In [5]:
import pandas as pd

In [15]:
os.path.join("..",*"Data EAF\material".split(os.path.sep),'ee')

'..\\Data EAF\\material\\ee'

In [56]:
class EafReader():
    def __init__(self,filename,path):
        self.filename = filename
        self.xtree = et.parse(os.path.join(*path.split(os.path.sep),self.filename))
        self.xroot = self.xtree.getroot()
    def parser(self,attribute):
        annot = []
        annot_df = []
        for el in self.xroot.findall('./TIER'):
            if el.attrib['TIER_ID'] == attribute:
                for k in el.findall('./ANNOTATION/ALIGNABLE_ANNOTATION'):
                    annot_df.append(k.attrib)
                    annot.append(k.find('ANNOTATION_VALUE').text)
        return annot, annot_df
    def dataframe_creator(self,annot,AnnotDf,annot_type='dstr'):
        annot_df = pd.DataFrame(AnnotDf)
        annot_df[annot_type] = annot
        annot_df = pd.merge(annot_df,data_time, 
                            left_on='TIME_SLOT_REF1',
                            right_on='TIME_SLOT_ID',
                            how='left')
        annot_df = annot_df.rename(columns={'TIME_VALUE':'Begin Time',
                                            'annotation':annot_type}).drop('TIME_SLOT_ID',axis=1)
        annot_df = pd.merge(annot_df,
                            data_time, 
                            left_on='TIME_SLOT_REF2',
                            right_on='TIME_SLOT_ID',
                            how='left').rename(columns={'TIME_VALUE':'End Time'})
        annot_df.drop(['TIME_SLOT_REF1','TIME_SLOT_REF2','TIME_SLOT_ID'],
                      axis=1,
                      inplace=True)
        annot_df[['Begin Time','End Time']] = annot_df[['Begin Time','End Time']].apply(lambda x: x.astype('int64'))
        annot_df['Nr'] = annot_df.index +1
        return annot_df

In [57]:
rdr = EafReader('G01V01P01_L.eaf','..\materials')

In [58]:
dstr_annot, dstr_annot_df = rdr.parser('dstr')
dstr_ = rdr.dataframe_creator(dstr_annot,dstr_annot_df)
dstr_

,ANNOTATION_ID,CVE_REF,dstr,Begin Time,End Time,Nr
0,a10907,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,1544,1749,1
1,a14736,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,2839,3012,2
2,a14737,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,4635,4903,3
3,a14738,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,10408,10789,4
4,a14740,cveid_982705de-1b7c-464b-9b2e-f82d34b08020,DEL,17152,18151,5
...,...,...,...,...,...,...
340,a16133,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,682244,682577,341
341,a16134,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,684170,684440,342
342,a16136,cveid_1ada0200-ccfb-4bf1-a447-f998e1f5f56e,REP,684760,685031,343
343,a16138,cveid_9ba87e8c-e6e7-45b2-9728-4310820223ce,FP,687010,687420,344


In [59]:

dstr_annot, dstr_annot_df = rdr.parser('ORT')
ort_ = rdr.dataframe_creator(dstr_annot,dstr_annot_df,annot_type='ORT')
ort_

> c:\users\hellf\.conda\envs\tacotron2\lib\site-packages\pandas\core\indexes\base.py(2900)get_loc()
   2898                 return self._engine.get_loc(casted_key)
   2899             except KeyError as err:
-> 2900                 raise KeyError(key) from err
   2901 
   2902         if tolerance is not None:



ipdb>  step


,ANNOTATION_ID,ORT,Begin Time,End Time,Nr
0,a26220,Buongiorno,644,1030,1
1,a26221,a,1030,1080,2
2,a26222,tutti <ii>,1080,1750,3
3,a26223,ragazzi,1750,2560,4
4,a26224,<sp>,2560,2670,5
...,...,...,...,...,...
1832,a28054,per,690754,690910,1833
1833,a28055,visitare,690910,691576,1834
1834,a28056,<sp>,691576,691659,1835
1835,a28057,il,691659,691777,1836


Da aggiungere la funzione insert_indf

In [60]:
def conciliator(annot_df, ort_df):
    import numpy as np
    conciliation = []
    for cold, rowd in annot_df.iterrows():
        for colo, rowo in ort_df.iterrows():
            if rowd['Begin Time'] > rowo['Begin Time'] and rowd['Begin Time'] < rowo['End Time']:
                conciliation.append({'conc_ort':rowo['Nr'],
                                     'conc_dstr':rowd['Nr'],
                                     'beg_tm_ort':rowo['Begin Time'],
                                     'end_tm_ort':rowo['End Time'],
                                     'beg_tm_dstr':rowd['Begin Time'],
                                     'end_tm_dstr':rowd['End Time'],
                                     'dstr':rowd['dstr'],
                                     'ort':rowo['ORT']})
    conc_data = pd.DataFrame(conciliation)
    return conc_data

In [61]:
conc_data = conciliator(dstr_,ort_)

In [62]:
conc_data

,conc_ort,conc_dstr,beg_tm_ort,end_tm_ort,beg_tm_dstr,end_tm_dstr,dstr,ort
0,3,1,1080,1750,1544,1749,PRL,tutti <ii>
1,6,2,2670,3010,2839,3012,PRL,io<oo>
2,10,3,4360,4900,4635,4903,PRL,sono<oo>
3,23,4,9818,10790,10408,10789,PRL,Certosa<aa>
4,45,6,18150,18254,18151,18251,FP,<eeh>
...,...,...,...,...,...,...,...,...
156,1804,339,679690,679770,679691,680125,LFP,in
157,1813,341,682119,682577,682244,682577,PRL,e<ee>
158,1818,342,683810,684440,684170,684440,PRL,sue<ee>
159,1826,344,686992,687420,687010,687420,FP,<ehm>


In [36]:
def insert_indf(df,match_rule,df_to_insert_pre,df_to_insert_post,subst = False):
    index = df.loc[match_rule,:].index
    for el in index:
        if not subst:
            upp_df = df.loc[:el,:]
            low_df = df.loc[el+1:,:]
        else:
            upp_df = df.loc[:el-1,:]
            low_df = df.loc[el+1:,:]
        df = pd.concat([upp_df,df_to_insert_pre.loc[el],df_to_insert_post.loc[el],low_df])
    #df = df.reset_index(drop=True)
    return df

Questa sopra è la versione semplice, la versione complessa dovrebbe riguardare il prendere il data frame ort, fare il match per regex e poi riempire lo stesso con i valori del match per regex del data frame conc.

In [37]:
values = ort_df.loc[ort_df.ORT.str.match('\w+\s*\<.*\>'),:].ORT.str.split('\s*\<|\>',expand=True).iloc[:,1:]

In [38]:
disfl = '<'+values.iloc[:,0].astype('str')+'>'

In [40]:
post = values.iloc[:,1]

In [41]:
disfl = disfl.to_frame(name='ORT')

In [42]:
post = post.replace({'':np.nan,' ':np.nan})
post = post.to_frame(name='ORTP')

In [43]:
ort_df

,ANNOTATION_ID,ORT,Begin Time,End Time,Nr
0,a26220,Buongiorno,644,1030,1
1,a26221,a,1030,1080,2
2,a26222,tutti <ii>,1080,1750,3
3,a26223,ragazzi,1750,2560,4
4,a26224,<sp>,2560,2670,5
...,...,...,...,...,...
1832,a28054,per,690754,690910,1833
1833,a28055,visitare,690910,691576,1834
1834,a28056,<sp>,691576,691659,1835
1835,a28057,il,691659,691777,1836


Ci sono anche dei valori dopo la correzione. Però per quello che dobbiamo fare ora, concentriamoci sul caso semplice, ovvero con il pattern: parola\<prolungamento\>

In [44]:
ort_df.loc[:10,:]

,ANNOTATION_ID,ORT,Begin Time,End Time,Nr
0,a26220,Buongiorno,644,1030,1
1,a26221,a,1030,1080,2
2,a26222,tutti <ii>,1080,1750,3
3,a26223,ragazzi,1750,2560,4
4,a26224,<sp>,2560,2670,5
5,a26225,io<oo>,2670,3010,6
6,a26226,sono,3010,3250,7
7,a26227,Camilla,3250,3730,8
8,a26228,<sp>,3730,4360,9
9,a26229,sono<oo>,4360,4900,10


In [45]:
ort_data_new = insert_indf(ort_df,ort_df.ORT.str.match('\w+\s*\<.*\>'),disfl,post)

In [46]:
ort_data_new = ort_data_new.dropna(how='all')

In [47]:
ort_data_new.Nr=ort_data_new.Nr.fillna(method='ffill')

In [48]:
ort_data_new.Nr = ort_data_new.Nr.astype('int')

In [49]:
ort_data_new=ort_data_new.reset_index(drop=True)

In [50]:
ort_data_new.head()

,0,ANNOTATION_ID,Begin Time,End Time,Nr,ORT
0,NaN,a26220,644.0,1030.0,1,Buongiorno
1,NaN,a26221,1030.0,1080.0,2,a
2,NaN,a26222,1080.0,1750.0,3,tutti <ii>
3,<ii>,NaN,NaN,NaN,3,NaN
4,NaN,a26223,1750.0,2560.0,4,ragazzi


In [55]:
beg_tm = []
end_tm = []
for idx in ort_data_new.index:
    if pd.isnull(ort_data_new.loc[idx,'Begin Time']):
        if conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values.shape[0] == 1:
            ort_data_new.loc[idx-1,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'Begin Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'end_tm_ort'].values
        elif conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values.shape[0] > 1:
            ort_data_new.loc[idx-1,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr'])&(conc_data.dstr=='PRL'),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'Begin Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr'])&(conc_data.dstr=='PRL'),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr'])&(conc_data.dstr=='PRL'),'end_tm_ort'].values

In [66]:
pd.options.display.max_rows = 999

In [67]:
ort_data_new[~ort_data_new.loc[:,0].isna()]

,0,ANNOTATION_ID,Begin Time,End Time,Nr,ORT
3,<ii>,NaN,1544.0,1750.0,3,NaN
7,<oo>,NaN,2839.0,3010.0,6,NaN
12,<oo>,NaN,4635.0,4900.0,10,NaN
26,<aa>,NaN,10408.0,10790.0,23,NaN
61,<ee>,NaN,23353.0,23642.0,57,NaN
66,<ii>,NaN,24783.0,24970.0,61,NaN
87,<ii>,NaN,31823.0,32020.0,81,NaN
101,<aa>,NaN,36542.0,36771.0,94,NaN
118,<oo>,NaN,43388.0,43570.0,110,NaN
131,<aa>,NaN,46698.0,46860.0,122,NaN


Semrba bloccarsi qui, scopri il perchè! Elan ha inserito un <aa> in marmi al secondo 431510 alla cavolo!

In [54]:
conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),:]

,conc_ort,conc_dstr,beg_tm_ort,end_tm_ort,beg_tm_dstr,end_tm_dstr,dstr,ort
54,454,93,177850,178480,177851,178293,LFP,quindi<ii>
55,454,94,177850,178480,178293,178480,PRL,quindi<ii>


In [53]:
ort_data_new.loc[idx-3:idx+2,'Begin Time']

306    110550.0
307    110930.0
308    111010.0
309         NaN
310    111780.0
311    112390.0
Name: Begin Time, dtype: float64

In [57]:
display(ort_df[ort_df['Begin Time']>=111010.0].head(10))
display(annot_df[annot_df['Begin Time']>=111010.0].head(10))

,ANNOTATION_ID,ORT,Begin Time,End Time,Nr
287,a26507,questa<aa>,111010,111780,288
288,a26508,gloriosa,111780,112390,289
289,a26509,per,112390,112522,290
290,a26510,l'appunto,112522,112970,291
291,a26511,tradizione<ee>,112970,114150,292
292,a26512,<sp>,114150,114280,293
293,a26513,napoletana,114280,115020,294
294,a26514,<sp>,115020,116115,295
295,a26515,<tongue-click>,116115,116187,296
296,a26516,<sp>,116187,116480,297


,ANNOTATION_ID,CVE_REF,dstr,Begin Time,End Time,Nr
54,a14802,cveid_9ba87e8c-e6e7-45b2-9728-4310820223ce,FP,111345,111780,55
55,a14803,cveid_c6187704-5d40-4a03-a29c-4a1b86453cdd,LFP,112393,112967,56
56,a14804,cveid_d673a93d-9f59-44c4-b1a1-6c7e214c6d6f,PRL,113698,114150,57
57,a30212,cveid_28867b92-92a6-4bb8-9ad8-665c02a73a98,SP,114150,114280,58
58,a14805,cveid_28867b92-92a6-4bb8-9ad8-665c02a73a98,SP,116960,117120,59
59,a14806,cveid_9ba87e8c-e6e7-45b2-9728-4310820223ce,FP,117120,117412,60
60,a14807,cveid_28867b92-92a6-4bb8-9ad8-665c02a73a98,SP,117412,117596,61
61,a14808,cveid_c6187704-5d40-4a03-a29c-4a1b86453cdd,LFP,118073,118610,62
62,a14810,cveid_28867b92-92a6-4bb8-9ad8-665c02a73a98,SP,124130,124540,63
63,a14811,cveid_c6187704-5d40-4a03-a29c-4a1b86453cdd,LFP,124540,125100,64


In [54]:
conc_data.loc[conc_data.conc_ort >= 288,:]

,conc_ort,conc_dstr,beg_tm_ort,end_tm_ort,beg_tm_dstr,end_tm_dstr,dstr,ort
31,288,55,111010,111780,111345,111780,FP,questa<aa>
32,290,56,112390,112522,112393,112967,LFP,per
33,292,57,112970,114150,113698,114150,PRL,tradizione<ee>
34,303,62,118070,118250,118073,118610,LFP,in
35,322,65,125100,125690,125359,125696,PRL,della<aa>
...,...,...,...,...,...,...,...,...
155,1804,338,679690,679770,679691,680125,LFP,in
156,1813,340,682119,682577,682244,682577,PRL,e<ee>
157,1818,341,683810,684440,684170,684440,PRL,sue<ee>
158,1826,343,686992,687420,687010,687420,FP,<ehm>


1190

In [ ]:
ort_data_new

In [ ]:
pd.merge(ort_data_new,conc_data, left_on = 'Nr', right_on='conc_ort',how='left')

### Nota

Qui potremmo pensare se inserire il token della disfluenza o meno, per vedere se la rete allunga l'ultima vocale della parola indicata. Alcune parole hanno la disfluenza internamente, ovvero dentro le stesse.

Per fare il fillna, quindi possiamo:
- Usare il token PRL;
- Usare il token specifico inerente la vocale che viene allungata.



In [ ]:
ort_data_new.ORT = ort_data_new.ORT.fillna(ort_data_new.iloc[:,0].dropna())

In [ ]:
ort_data_new

In [ ]:
ort_data_new.ORT = ort_data_new.ORT.str.replace(r'(\w+)(\s)(\<.*\>)',r'\1')

In [ ]:
ort_data_new.ORT = ort_data_new.ORT.str.replace(r'(\w+)(\<.*\>)',r'\1')
ort_data_new = ort_data_new.drop(0,axis=1)

In [ ]:
ort_data_new

In [ ]:
ort_data_new.Duration=ort_data_new.Duration.fillna(ort_data_new['End Time']-ort_data_new['Begin Time'])

In [ ]:
ort_data_new.to_csv('data_completed2.csv')

In [ ]:
annot_df

In [ ]:
import seaborn as sns

sns.heatmap()

In [ ]:
annot_df.info()

---

Da qui leggiamo i files csv!

In [68]:
import pandas as pd
pd.options.display.max_rows = 999

In [69]:
data1 = pd.read_csv('annotations.csv')
data2 = pd.read_csv('timeslots.csv')

In [70]:
display(data1)
display(data2)

,Unnamed: 0,ANNOTATION_ID,CVE_REF,TIME_SLOT_REF1,TIME_SLOT_REF2,LINGUISTIC_TYPE_REF,TIER_ID,PARENT_REF,annotation
0,0,a10904,cveid_0539abdc-6bc6-4b41-b35f-0423dd889dba,ts39,ts50,dml,dml,NaN,OU
1,1,a10905,cveid_25e59bf3-657c-4d02-9ba2-23b9abded943,ts48,ts54,dml,dml,NaN,IM
2,2,a10906,cveid_a319e893-3eda-470a-9b77-4feae2156020,ts56,ts93,dml,dml,NaN,CNT
3,3,a14550,cveid_0539abdc-6bc6-4b41-b35f-0423dd889dba,ts101,ts106,dml,dml,NaN,OU
4,4,a14551,cveid_25e59bf3-657c-4d02-9ba2-23b9abded943,ts105,ts122,dml,dml,NaN,IM
...,...,...,...,...,...,...,...,...,...
10244,10244,a39012,NaN,ts19773,ts19778,default-lt,phn,NaN,-a$bb
10245,10245,a39013,NaN,ts19779,ts19782,default-lt,phn,NaN,ja~
10246,10246,a39014,NaN,ts19783,ts19785,default-lt,phn,NaN,m
10247,10247,a39015,NaN,ts19786,ts19797,default-lt,phn,NaN,O~


,Unnamed: 0,TIME_SLOT_ID,TIME_VALUE
0,0,ts1,644.0
1,1,ts2,644.0
2,2,ts3,644.0
3,3,ts4,644.0
4,4,ts5,644.0
...,...,...,...
30509,30509,ts30510,692170.0
30510,30510,ts30511,692170.0
30511,30511,ts30512,692170.0
30512,30512,ts30513,692171.0


## Dataset creation

In questo dataset dobbiamo creare un match tra le variabili tali per cui:
- ORT ci da la trascrizione
- dstr ci da la parte di disfluenze.

In [1]:
import pandas as pd

In [2]:
ort_data = pd.read_csv('G01V01P01_ort.txt',sep='\t')
dstr_data = pd.read_csv('G01V01P01_dstr.txt',sep='\t')

In [3]:
display(ort_data.head())
display(dstr_data.head())

,Nr,ORT,Begin Time,End Time,Duration,Unnamed: 5
0,1,Buongiorno,644,1030,386,NaN
1,2,a,1030,1080,50,NaN
2,3,tutti <ii>,1080,1750,670,NaN
3,4,ragazzi,1750,2560,810,NaN
4,5,<sp>,2560,2670,110,NaN


,Nr,dstr,Begin Time,End Time,Duration,Unnamed: 5
0,1,PRL,1544,1749,205,NaN
1,2,PRL,2839,3012,173,NaN
2,3,PRL,4635,4903,268,NaN
3,4,PRL,10408,10789,381,NaN
4,5,DEL,17152,18151,999,NaN


In [4]:
ort_data = ort_data.dropna(how='all',axis = 1)
dstr_data = dstr_data.dropna(how='all',axis = 1)

In [5]:
display(ort_data.head())
display(dstr_data.head())

,Nr,ORT,Begin Time,End Time,Duration
0,1,Buongiorno,644,1030,386
1,2,a,1030,1080,50
2,3,tutti <ii>,1080,1750,670
3,4,ragazzi,1750,2560,810
4,5,<sp>,2560,2670,110


,Nr,dstr,Begin Time,End Time,Duration
0,1,PRL,1544,1749,205
1,2,PRL,2839,3012,173
2,3,PRL,4635,4903,268
3,4,PRL,10408,10789,381
4,5,DEL,17152,18151,999


In [6]:
dstr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nr          345 non-null    int64 
 1   dstr        345 non-null    object
 2   Begin Time  345 non-null    int64 
 3   End Time    345 non-null    int64 
 4   Duration    345 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 13.6+ KB


In [7]:
ort_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837 entries, 0 to 1836
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nr          1837 non-null   int64 
 1   ORT         1837 non-null   object
 2   Begin Time  1837 non-null   int64 
 3   End Time    1837 non-null   int64 
 4   Duration    1837 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 71.9+ KB


## TODO:

Nel caso delle disfluenze in ort, queste vanno scomposte d'accordo con dstr. Si prenda ad esempio la riga 2 del ort_data: in questo caso **"ii"** va spostato in una riga di sotto, andando a prendere l'intervallo da dstr. In particolare, l'intervallo della riga 2 diventerebbe:

|ORT|Begin Time|End Time|
|---|---|---|
|tutti | 1080 | 1544|
|PRL | 1544 | 1750|

Prove da fare:

1. inserisci il token della disfluenza da dstr in modo da vedere se la rete riesce a replicare la disfluenza indipendentemente dalla stessa, prolungando semplicemente l'ultima lettera
2. inserisci il token della disfluenza tra parentesi quadre.

In [8]:
match_disfl_alone = ort_data.ORT.str.match('<.*>')

In [9]:
match_disfl_alone

0       False
1       False
2       False
3       False
4        True
        ...  
1832    False
1833    False
1834     True
1835    False
1836    False
Name: ORT, Length: 1837, dtype: bool

In [10]:
pd.set_option("display.max_rows", 9999)

In [11]:
match_disfl_within = ort_data.ORT.str.match("[a-zA-Z]+\s*\<.*\>")

In [12]:
import numpy as np
conciliation = []
for cold, rowd in dstr_data.iterrows():
    for colo, rowo in ort_data.iterrows():
        if rowd['Begin Time'] > rowo['Begin Time'] and rowd['Begin Time'] < rowo['End Time']:
            conciliation.append({'conc_ort':rowo['Nr'],
                                 'conc_dstr':rowd['Nr'],
                                 'beg_tm_ort':rowo['Begin Time'],
                                 'end_tm_ort':rowo['End Time'],
                                 'beg_tm_dstr':rowd['Begin Time'],
                                 'end_tm_dstr':rowd['End Time'],
                                 'dstr':rowd['dstr'],
                                 'ort':rowo['ORT']})

In [13]:
#import sqlite3

In [14]:
conc_data = pd.DataFrame(conciliation)

In [15]:
conc_data

,conc_ort,conc_dstr,beg_tm_ort,end_tm_ort,beg_tm_dstr,end_tm_dstr,dstr,ort
0,3,1,1080,1750,1544,1749,PRL,tutti <ii>
1,6,2,2670,3010,2839,3012,PRL,io<oo>
2,10,3,4360,4900,4635,4903,PRL,sono<oo>
3,23,4,9818,10790,10408,10789,PRL,Certosa<aa>
4,45,6,18150,18254,18151,18251,FP,<eeh>
5,57,8,23180,23642,23353,23646,PRL,che<ee>
6,61,9,24154,24970,24783,24970,PRL,ragazzi<ii>
7,81,11,31670,32020,31823,32021,PRL,di<ii>
8,94,13,36040,36771,36542,36771,FP,tanta<aa>
9,110,15,43260,43570,43388,43570,PRL,o<oo>


In [16]:
conc_data.to_csv('conciliation_table2.csv')

In [17]:
ort_data.head(10)

,Nr,ORT,Begin Time,End Time,Duration
0,1,Buongiorno,644,1030,386
1,2,a,1030,1080,50
2,3,tutti <ii>,1080,1750,670
3,4,ragazzi,1750,2560,810
4,5,<sp>,2560,2670,110
5,6,io<oo>,2670,3010,340
6,7,sono,3010,3250,240
7,8,Camilla,3250,3730,480
8,9,<sp>,3730,4360,630
9,10,sono<oo>,4360,4900,540


In [18]:
toy_df = pd.DataFrame([{'aa':1,'bb':1},{'aa':2,'bb':1},{'aa':1,'bb':1}])

In [19]:
def insert_indf(df,match_rule,df_to_insert_pre,df_to_insert_post,subst = False):
    index = df.loc[match_rule,:].index
    for el in index:
        if not subst:
            upp_df = df.loc[:el,:]
            low_df = df.loc[el+1:,:]
        else:
            upp_df = df.loc[:el-1,:]
            low_df = df.loc[el+1:,:]
        df = pd.concat([upp_df,df_to_insert_pre.loc[el],df_to_insert_post.loc[el],low_df])
    #df = df.reset_index(drop=True)
    return df

Questa sopra è la versione semplice, la versione complessa dovrebbe riguardare il prendere il data frame ort, fare il match per regex e poi riempire lo stesso con i valori del match per regex del data frame conc.

In [20]:
values = ort_data.loc[ort_data.ORT.str.match('\w+\s*\<.*\>'),:].ORT.str.split('\s*\<|\>',expand=True).iloc[:,1:] #controlla che su dstr corrisponda ad un PRL

In [21]:
disfl = '<'+values.iloc[:,0].astype('str')+'>'

In [22]:
disfl

2       <ii>
5       <oo>
9       <oo>
22      <aa>
56      <ee>
60      <ii>
80      <ii>
93      <aa>
109     <oo>
121     <aa>
128     <ii>
137     <aa>
146     <ii>
171     <ee>
172     <aa>
235     <aa>
248     <ii>
256     <ee>
267     <oo>
277     <ii>
279     <ii>
287     <aa>
291     <ee>
321     <aa>
324     <ee>
354     <aa>
361     <ee>
365     <ii>
368     <aa>
373     <aa>
382     <ee>
400     <oo>
406     <ee>
411     <ee>
412     <ee>
431     <aa>
434     <aa>
453     <ii>
495     <aa>
516     <aa>
521     <ee>
528     <aa>
536     <ee>
595     <ii>
671     <ee>
679     <aa>
695     <aa>
745     <oo>
761     <ee>
767     <ii>
830     <nn>
974     <oo>
1001    <aa>
1006    <oo>
1020    <oo>
1033    <ii>
1051    <oo>
1057    <uu>
1073    <ee>
1081    <ii>
1128    <aa>
1135    <oo>
1165    <oo>
1176    <ii>
1184    <oo>
1195    <aa>
1206    <oo>
1211    <aa>
1214    <ii>
1215    <aa>
1233    <ee>
1245    <oo>
1247    <ll>
1249    <aa>
1273    <nn>
1278    <aa>
1328    <aa>

In [23]:
post = values.iloc[:,1]

In [24]:
disfl = disfl.to_frame(name='ORT')

In [25]:
post = post.replace({'':np.nan,' ':np.nan})
post = post.to_frame(name='ORTP')

Ci sono anche dei valori dopo la correzione. Però per quello che dobbiamo fare ora, concentriamoci sul caso semplice, ovvero con il pattern: parola\<prolungamento\>

In [26]:
ort_data_new = pd.DataFrame()
ort_data_new = insert_indf(ort_data,ort_data.ORT.str.match('\w+\s*\<.*\>'),disfl,post)

In [27]:
ort_data_new = ort_data_new.dropna(how='all')

In [28]:
ort_data_new.Nr=ort_data_new.Nr.fillna(method='ffill')

In [29]:
ort_data_new.Nr = ort_data_new.Nr.astype('int')

In [30]:
ort_data_new=ort_data_new.reset_index(drop=True)

In [31]:
ort_data_new

,0,Begin Time,Duration,End Time,Nr,ORT
0,NaN,644.0,386.0,1030.0,1,Buongiorno
1,NaN,1030.0,50.0,1080.0,2,a
2,NaN,1080.0,670.0,1750.0,3,tutti <ii>
3,<ii>,NaN,NaN,NaN,3,NaN
4,NaN,1750.0,810.0,2560.0,4,ragazzi
5,NaN,2560.0,110.0,2670.0,5,<sp>
6,NaN,2670.0,340.0,3010.0,6,io<oo>
7,<oo>,NaN,NaN,NaN,6,NaN
8,NaN,3010.0,240.0,3250.0,7,sono
9,NaN,3250.0,480.0,3730.0,8,Camilla


In [32]:
beg_tm = []
end_tm = []
for idx in ort_data_new.index:
    if pd.isnull(ort_data_new.loc[idx,'Begin Time']):
        if conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values.shape[0] == 1:
            ort_data_new.loc[idx-1,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'Begin Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'end_tm_ort'].values
        elif conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr']),'beg_tm_dstr'].values.shape[0] > 1:
            ort_data_new.loc[idx-1,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr'])&(conc_data.dstr=='PRL'),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'Begin Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr'])&(conc_data.dstr=='PRL'),'beg_tm_dstr'].values
            ort_data_new.loc[idx,'End Time'] = conc_data.loc[(conc_data.conc_ort == ort_data_new.loc[idx,'Nr'])&(conc_data.dstr=='PRL'),'end_tm_ort'].values

In [33]:
ort_data_new.loc[1821-1:1821+1,:]

,0,Begin Time,Duration,End Time,Nr,ORT
1820,NaN,648090.0,810.0,648900.0,1716,angeli<ii>
1821,<ii>,NaN,NaN,NaN,1716,NaN
1822,NaN,648900.0,210.0,649110.0,1717,<sp>


In [35]:
ort_data_new.dropna(subset=['Begin Time','End Time'],how='all',inplace = True)

In [37]:
ort_data_new['Duration'] = ort_data_new['End Time'] - ort_data_new['Begin Time']

In [38]:
ort_data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1934 entries, 0 to 1946
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   0           97 non-null     object 
 1   Begin Time  1934 non-null   float64
 2   Duration    1934 non-null   float64
 3   End Time    1934 non-null   float64
 4   Nr          1934 non-null   int32  
 5   ORT         1837 non-null   object 
dtypes: float64(3), int32(1), object(2)
memory usage: 98.2+ KB


## Strategy 1

Implementiamo il missing token nella colonna ORT con PRL

In [39]:
ort_data_new_prl = ort_data_new.copy()
ort_data_new_prl.ORT = ort_data_new_prl.ORT.fillna('PRL')

In [42]:
display(ort_data_new_prl.head())
ort_data_new_prl = ort_data_new_prl.loc[:,'Begin Time':]

,0,Begin Time,Duration,End Time,Nr,ORT
0,NaN,644.0,386.0,1030.0,1,Buongiorno
1,NaN,1030.0,50.0,1080.0,2,a
2,NaN,1080.0,464.0,1544.0,3,tutti <ii>
3,<ii>,1544.0,206.0,1750.0,3,PRL
4,NaN,1750.0,810.0,2560.0,4,ragazzi


In [46]:
ort_data_new_prl.ORT = ort_data_new_prl.ORT.str.replace(r'(\w+)(\s)(\<.*\>)',r'\1')
ort_data_new_prl.ORT = ort_data_new_prl.ORT.str.replace(r'(\w+)(\<.*\>)',r'\1')

C:\Users\hellf\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
C:\Users\hellf\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [47]:
ort_data_new_prl.head()

,Begin Time,Duration,End Time,Nr,ORT
0,644.0,386.0,1030.0,1,Buongiorno
1,1030.0,50.0,1080.0,2,a
2,1080.0,464.0,1544.0,3,tutti
3,1544.0,206.0,1750.0,3,PRL
4,1750.0,810.0,2560.0,4,ragazzi


## Strategy 2

integriamo i missing tokens con i brackets

In [43]:
ort_data_new_tok = ort_data_new.copy()
ort_data_new_tok.ORT = ort_data_new_tok.ORT.fillna(ort_data_new_tok.iloc[:,0].dropna())

In [45]:
ort_data_new_tok = ort_data_new_tok.loc[:,'Begin Time':]
ort_data_new_tok.head()

,Begin Time,Duration,End Time,Nr,ORT
0,644.0,386.0,1030.0,1,Buongiorno
1,1030.0,50.0,1080.0,2,a
2,1080.0,464.0,1544.0,3,tutti <ii>
3,1544.0,206.0,1750.0,3,<ii>
4,1750.0,810.0,2560.0,4,ragazzi


In [48]:
ort_data_new_tok.ORT = ort_data_new_tok.ORT.str.replace(r'(\w+)(\s)(\<.*\>)',r'\1')
ort_data_new_tok.ORT = ort_data_new_tok.ORT.str.replace(r'(\w+)(\<.*\>)',r'\1')

C:\Users\hellf\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
C:\Users\hellf\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [49]:
ort_data_new_tok.head()

,Begin Time,Duration,End Time,Nr,ORT
0,644.0,386.0,1030.0,1,Buongiorno
1,1030.0,50.0,1080.0,2,a
2,1080.0,464.0,1544.0,3,tutti
3,1544.0,206.0,1750.0,3,<ii>
4,1750.0,810.0,2560.0,4,ragazzi


---

# Other trials

In [41]:
ort_improved = ort_data_new.loc[~ort_data_new.loc[:,0].isna(),[0,'Begin Time','End Time','Nr','ORT']].dropna(thresh=4)
ort_improved['dstr'] = 'PRL'

In [45]:
pd.merge(ort_data_new,ort_improved[['Nr','dstr']],copy=False,how='left')

,0,Begin Time,Duration,End Time,Nr,ORT,dstr
0,NaN,644.0,386.0,1030.0,1,Buongiorno,NaN
1,NaN,1030.0,50.0,1080.0,2,a,NaN
2,NaN,1080.0,670.0,1544.0,3,tutti <ii>,PRL
3,<ii>,1544.0,NaN,1750.0,3,NaN,PRL
4,NaN,1750.0,810.0,2560.0,4,ragazzi,NaN
5,NaN,2560.0,110.0,2670.0,5,<sp>,NaN
6,NaN,2670.0,340.0,2839.0,6,io<oo>,PRL
7,<oo>,2839.0,NaN,3010.0,6,NaN,PRL
8,NaN,3010.0,240.0,3250.0,7,sono,NaN
9,NaN,3250.0,480.0,3730.0,8,Camilla,NaN


In [32]:
row.loc['Begin Time']

NameError: name 'row' is not defined

In [34]:
idx

1946

In [128]:
conc_data.loc[conc_data.conc_ort == ort_data_new.loc[idx,'Nr']]

,conc_ort,conc_dstr,beg_tm_ort,end_tm_ort,beg_tm_dstr,end_tm_dstr,dstr,ort
54,454,93,177850,178480,177851,178293,LFP,quindi<ii>
55,454,94,177850,178480,178293,178480,PRL,quindi<ii>


In [110]:
ort_data_new.loc[idx-1:idx+2]

,0,Begin Time,Duration,End Time,Nr,ORT
490,NaN,177850.0,630.0,178480.0,454,quindi<ii>
491,<ii>,NaN,NaN,NaN,454,NaN
492,NaN,178480.0,290.0,178770.0,455,il
493,NaN,178770.0,480.0,179250.0,456,figlio


In [ ]:
pd.merge(ort_data_new,conc_data, left_on = 'Nr', right_on='conc_ort',how='left')

### Nota

Qui potremmo pensare se inserire il token della disfluenza o meno, per vedere se la rete allunga l'ultima vocale della parola indicata. Alcune parole hanno la disfluenza internamente, ovvero dentro le stesse.

Per fare il fillna, quindi possiamo:
- Usare il token PRL;
- Usare il token specifico inerente la vocale che viene allungata.



In [ ]:
ort_data_new.ORT = ort_data_new.ORT.fillna(ort_data_new.iloc[:,0].dropna())

In [ ]:
ort_data_new

In [ ]:
ort_data_new.ORT = ort_data_new.ORT.str.replace(r'(\w+)(\s)(\<.*\>)',r'\1')

In [ ]:
ort_data_new.ORT = ort_data_new.ORT.str.replace(r'(\w+)(\<.*\>)',r'\1')
ort_data_new = ort_data_new.drop(0,axis=1)

In [ ]:
ort_data_new

In [ ]:
ort_data_new.Duration=ort_data_new.Duration.fillna(ort_data_new['End Time']-ort_data_new['Begin Time'])

In [ ]:
ort_data_new.to_csv('data_completed2.csv')